In [1]:
from stats import run_algo
from typing import List

In [2]:
filename = 'result.txt'

In [3]:
import os
run_algo(alg='CBS', dest_file='result.txt',
         map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', 'random-32-32-10.map'),
         scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', 'random-32-32-10-even-1.scen'),
         tasks_count=10,
         test_num=10)

True

In [4]:
from collections import defaultdict
def parse_path_file(path_file_content):
    agents_path = defaultdict(list)
    agent = None
    lines = path_file_content.split('\n')
    for line in lines:
        if line == '':
            break
        nums = list(map(int, line.split(' ')))
        if len(nums) == 1:
            agent = int(nums[0])
        else:
            x, y, time = nums
            agents_path[agent].append((x, y, time))
    return agents_path

def read_paths(filename=filename):
    with open(filename, 'r') as path_file:
        path_file_content = path_file.read()
        agents_path = parse_path_file(path_file_content)
    return dict(agents_path)

In [5]:
import tqdm
# test num takes index in range(0, num_repetitions)
def success_rate(alg_name, map_name, scen_name, num_agents, num_repetitions, w=None):
    rate = 0
    for i in tqdm.tqdm(range(num_repetitions)):
        done = run_algo(alg=alg_name, dest_file=filename,
                 map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', map_name),
                 scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', scen_name),
                 tasks_count=num_agents,
                 test_num=i, w=w)
        rate += int(done)
    return rate / num_repetitions


In [8]:
def gen_rate_list(alg:str, map_name:str, scen_name:str, actors_array:List[int], N=10, w=None) -> List[float]:
    rate_list = []
    for num_actors in actors_array:
        rate = success_rate(alg, map_name, scen_name, num_actors, N, w)
        print(f'Num actors :{num_actors}, Rate: {rate:.2f}%')
        rate_list.append(rate)
    return rate_list

In [9]:
# CBS random-32x32x10
gen_rate_list('CBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', [5, 10, 20, 30, 40, 50])

  0%|          | 0/10 [00:00<?, ?it/s]Num actors :5, Rate: 1.00%
Num actors :10, Rate: 1.00%
 20%|██        | 2/10 [02:02<08:09, 61.22s/it]


KeyboardInterrupt: 

In [ ]:
# ECBS random-32x32x10 w=1.5
gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', [5, 10, 20, 30, 40, 50], w=1.5)

In [ ]:
# ECBS random-32x32x10 w=1.5 more agents
gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen',[5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.5)

In [ ]:
# ECBS random-32x32x10 w=1.3 more agents
gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen',[5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.3)

In [ ]:
# ECBS random-32x32x10 w=1.1 more agents
gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen',[5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.1)

In [ ]:
# CBS brc202d
gen_rate_list('CBS', 'brc202d.map', 'brc202d.scen', [1, 3, 5, 7, 10])

In [ ]:
# ECBS brc202d w=1.5
gen_rate_list('ECBS', 'brc202d.map', 'brc202d.scen', [1, 3, 5, 7, 10], w=1.5)

In [ ]:
# ECBS brc202d w=1.3
gen_rate_list('ECBS', 'brc202d.map', 'brc202d.scen', [1, 3, 5, 7, 10], w=1.3)

In [ ]:
# ECBS brc202d w=1.1
gen_rate_list('ECBS', 'brc202d.map', 'brc202d.scen', [1, 3, 5, 7, 10], w=1.1)

In [ ]:
# CBS den520d 
gen_rate_list('CBS', 'den520d.map', 'den520d.scen', [5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70])

In [ ]:
# ECBS den520d w=1.5
gen_rate_list('ECBS', 'den520d.map', 'den520d.scen', [5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.5)

In [ ]:
# ECBS den520d w=1.3
gen_rate_list('ECBS', 'den520d.map', 'den520d.scen', [5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.3)

In [ ]:
# ECBS den520d w=1.1
gen_rate_list('ECBS', 'den520d.map', 'den520d.scen', [5, 7, 10, 13, 16, 20, 23, 26, 30, 34, 37, 40, 43, 47, 50, 53, 56, 60, 63, 66, 70], w=1.1)